In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tf-nightly-gpu-2.0-preview

In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Convolution2D, Reshape
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization

from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

print(tf.__version__)

2.0.0-dev20190531


In [3]:
tf.test.is_gpu_available()

True

In [4]:
import handshape_datasets as hd

DATASET_NAME = "lsa16"

data = hd.load(DATASET_NAME)

In [5]:
!pip install -q -U scikit-learn
from sklearn.model_selection import train_test_split

features = data[0]
labels = data[1]['y']

x_train, x_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
labels.shape

(800,)

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0)
datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
test_datagen.fit(x_train)

In [8]:
#train_ds = tf.data.Dataset.from_tensor_slices(
#    (x_train, y_train)).shuffle(10000).batch(32)
#test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [9]:
def denseNet_model(growth_rate=32, nb_filter=64, nb_layers = [6,12,24,16], reduction=0.0, 
             dropout_rate=0.0, weight_decay=1e-4, classes=1000, batch_size=32, with_se_layers=True):
    
    with_se_layers = with_se_layers

    # compute compression factor
    compression = 1.0 - reduction

    nb_dense_block = len(nb_layers)
    # From architecture for ImageNet (Table 1 in the paper)
    # nb_filter = 64
    # nb_layers = [6,12,24,16] # For DenseNet-121
    
    img_input = Input(shape=(32, 32, 3), name='data')
    
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding', 
                  input_shape=(32, 32, 3), batch_size=batch_size)(img_input)
    x = Convolution2D(nb_filter, 7, 2, name='conv1', use_bias=False)(x)
    x = BatchNormalization(name='conv1_bn')(x)
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)
    
    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

        if (with_se_layers):
            x = se_block(x, stage, 'dense', nb_filter)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

        if (with_se_layers):
            x = se_block(x, stage, 'transition', nb_filter)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    if (with_se_layers):
        x = se_block(x, final_stage, 'dense', nb_filter)

    x = BatchNormalization(name='conv_final_blk_bn')(x)
    x = Activation('relu', name='relu_final_blk')(x)
    x = GlobalAveragePooling2D(name='pool_final')(x)
    x = Dense(classes, name='fc6')(x)
    output = Activation('softmax', name='prob')(x)
    
    return Model(inputs=img_input, outputs=output)

def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(name=conv_name_base+'_x1_bn')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(name=conv_name_base+'_x2_bn')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Convolution2D(nb_filter, 3, 1, name=conv_name_base+'_x2', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    return x

def se_block(x, stage, previous, nb_filter, ratio = 16):
    se_name = 'se' + str(stage) + '_' + previous
    init = x
    x = GlobalAveragePooling2D(name='global_average_pooling_2d_'+se_name)(x)
    x = Dense(nb_filter // ratio, name='dense_relu_'+se_name)(x)
    x = Activation('relu', name='relu_'+se_name)(x)
    x = Dense(nb_filter, name='dense_sigmoid_'+se_name)(x)
    x = Activation('sigmoid', name='sigmoid_'+se_name)(x)
    x = tf.expand_dims(x,1)
    x = init * tf.expand_dims(x,1) 
    return x

def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, 
                grow_nb_filters=True):
    concat_feat = x
    for i in range(nb_layers):
        branch = i+1
        x = conv_block(x, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = tf.concat([concat_feat, x], -1)

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage) 

    x = BatchNormalization(name=conv_name_base+'_bn')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x

model = denseNet_model(classes=16)
# model.load_weights(weights_path) 

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [12]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32), training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32), training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [14]:
EPOCHS = 800
models_directory = 'trained-models'
results_directory = 'results'
results = 'epoch,loss,accuracy,test_loss,test_accuracy\n'
from datetime import datetime
now = datetime.now()

log_freq = 10
save_freq = 100
identifier = now.strftime("%Y_%m_%d-%H:%M:%S")
train_summary_writer = tf.summary.create_file_writer('summaries/train/'+ identifier)
test_summary_writer = tf.summary.create_file_writer('summaries/test/'+ identifier)

for epoch in range(EPOCHS):

    batches = 0
    for images, labels in datagen.flow(x_train, y_train, batch_size=32):
        train_step(images, labels)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    batches = 0
    for test_images, test_labels in test_datagen.flow(x_test, y_test, batch_size=32, shuffle=False):
        test_step(test_images, test_labels)
        batches += 1
        if batches >= len(x_test) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    if (epoch % log_freq == 0):
        results += '{},{},{},{},{}\n'.format(epoch+1,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100)
        
        print ('Epoch: {}, Train Loss: {}, Train Acc:{}, Test Loss: {}, Test Acc: {}'.format(epoch+1,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100))

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
            train_loss.reset_states()           
            train_accuracy.reset_states()           

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            test_loss.reset_states()           
            test_accuracy.reset_states()           
        
        
    
    if (epoch % save_freq == 0):
        if not os.path.exists(models_directory):
            os.makedirs(models_directory)
        # serialize weights to HDF5
        model.save_weights(models_directory+"/dropout_se_model{}_epoch{}.h5".format(identifier,epoch))
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
file = open(results_directory+'/results'+ identifier + '.csv','w') 
file.write(results) 
file.close()

Epoch: 1, Train Loss: 3.043458938598633, Train Acc:7.462686538696289, Test Loss: 2.7706422805786133, Test Acc: 8.333333969116211
Epoch: 11, Train Loss: 2.2388598918914795, Train Acc:29.6641788482666, Test Loss: 2.798067331314087, Test Acc: 5.909090995788574
Epoch: 21, Train Loss: 1.5411089658737183, Train Acc:51.25, Test Loss: 2.986360788345337, Test Acc: 6.25
Epoch: 31, Train Loss: 1.1887325048446655, Train Acc:62.05223846435547, Test Loss: 2.3324947357177734, Test Acc: 24.772727966308594
Epoch: 41, Train Loss: 0.9962700605392456, Train Acc:68.50746154785156, Test Loss: 1.281662106513977, Test Acc: 60.871212005615234
Epoch: 51, Train Loss: 0.9263520836830139, Train Acc:71.1380615234375, Test Loss: 2.57774019241333, Test Acc: 55.30303192138672
Epoch: 61, Train Loss: 0.8348059058189392, Train Acc:74.60820770263672, Test Loss: 2.8631341457366943, Test Acc: 57.08333206176758
Epoch: 71, Train Loss: 0.77674400806427, Train Acc:77.55596923828125, Test Loss: 2.7536516189575195, Test Acc: 58.2

Epoch: 631, Train Loss: 0.10089632123708725, Train Acc:96.71641540527344, Test Loss: 0.5443257689476013, Test Acc: 86.78030395507812
Epoch: 641, Train Loss: 0.09585300832986832, Train Acc:96.75373077392578, Test Loss: 0.6266719698905945, Test Acc: 84.6212158203125
Epoch: 651, Train Loss: 0.08581169694662094, Train Acc:97.14552307128906, Test Loss: 0.49508875608444214, Test Acc: 87.46212005615234
Epoch: 661, Train Loss: 0.09028290957212448, Train Acc:96.95895385742188, Test Loss: 0.3908652365207672, Test Acc: 87.84091186523438
Epoch: 671, Train Loss: 0.08635737001895905, Train Acc:96.95895385742188, Test Loss: 0.7959036827087402, Test Acc: 81.74242401123047
Epoch: 681, Train Loss: 0.07465259730815887, Train Acc:97.5, Test Loss: 0.5715638399124146, Test Acc: 86.06060791015625
Epoch: 691, Train Loss: 0.07253492623567581, Train Acc:97.64925384521484, Test Loss: 0.4582061469554901, Test Acc: 87.42424011230469
Epoch: 701, Train Loss: 0.0960066169500351, Train Acc:96.69776153564453, Test Loss